In [1]:
import xml.etree.ElementTree as ET
import os as os
from PIL import Image
import numpy as np
import torch
import os
import cv2
import torch
import random
import numbers
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import randperm
from torch._utils import _accumulate
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as F
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from skimage import io, transform
import sklearn.metrics as skm
from scipy.spatial import distance
import import_ipynb 
from Utils import *
from Transformation import *

importing Jupyter notebook from Utils.ipynb
importing Jupyter notebook from Transformation.ipynb


In [2]:
class BlobTrainDataset(Dataset):
    
    def __init__(self, path, transform = None):
        
        self.path = path 
        self.transform = transform
        self.file_list = []
        self.xml_path = ""
        for file in os.listdir(path):
            #if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            if(file.endswith(".png")):
                self.xml_path = file.replace(".png",".xml")
            elif(file.endswith(".jpg")):
                self.xml_path = file.replace(".jpg",".xml")
            elif(file.endswith(".jpeg")):
                self.xml_path = file.replace(".jpeg",".xml")
                
            if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
                #print(os.path.join(self.path, self.xml_path))
                if os.path.exists(os.path.join(self.path, self.xml_path)):
                    self.file_list.append(file)
                
    def __len__(self):
        
        return len(self.file_list)
    
    def __getitem__(self, index):
        
        path = os.path.join(self.path, self.file_list[index])
        #print(self.file_list[index])
        image = Image.open(path)
        
        if(path.endswith(".png")):
            annotation = ET.parse(path.replace(".png",".xml")).getroot()
        elif(path.endswith(".jpg")):
            annotation = ET.parse(path.replace(".jpg",".xml")).getroot()
        elif(path.endswith(".jpeg")):
            annotation = ET.parse(path.replace(".jpeg",".xml")).getroot()
            
        anno_detail = []
        
        for element in annotation.findall("object"):
            anno_bound_box = element.find("bndbox")
            anno_detail.append(get_bounding_box(anno_bound_box))
        
        data_dict = {"image" : image, "data" : anno_detail}
        
        if(self.transform):
            
            if(type(self.transform) is not list):
                self.transform = [self.transform]
            
            for idx in range(len(self.transform)):
                data_dict = self.transform[idx](data_dict)
        
        image_size = data_dict["image"].shape
        
        heatmap_placeholder = torch.zeros([3, int(image_size[1]/4), int(image_size[2]/4)])
        
        object_counter = 0
        
        for obj in annotation.findall("object"):
            
            label = obj.find("name").text
            
            image_heatmap = heatmap(data_dict["data"][object_counter],(image_size[1],image_size[2]))
            
            if(label == "ball"):
                heatmap_placeholder[0] += image_heatmap
            elif (label == "goalpost"):
                heatmap_placeholder[1] += image_heatmap 
            elif (label == "robot"):
                heatmap_placeholder[2] += image_heatmap 
            
            object_counter+=1
            
        dataset = {'image': data_dict['image'], 'heatmap': heatmap_placeholder}
        return dataset

In [3]:
class BlobTestDataset(Dataset):
    
    def __init__(self, path, transform = None):
        
        self.path = path 
        self.transform = transform
        self.file_list = []
        self.xml_path = ""
        for file in os.listdir(path):
            #if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            if(file.endswith(".png")):
                self.xml_path = file.replace(".png",".xml")
            elif(file.endswith(".jpg")):
                self.xml_path = file.replace(".jpg",".xml")
            elif(file.endswith(".jpeg")):
                self.xml_path = file.replace(".jpeg",".xml")
                
            if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
                #print(os.path.join(self.path, self.xml_path))
                if os.path.exists(os.path.join(self.path, self.xml_path)):
                    self.file_list.append(file)
                
    def __len__(self):
        
        return len(self.file_list)
    
    def __getitem__(self, index):
        
        path = os.path.join(self.path, self.file_list[index])
        #print(self.file_list[index])
        image = Image.open(path)
        
        if(path.endswith(".png")):
            annotation = ET.parse(path.replace(".png",".xml")).getroot()
        elif(path.endswith(".jpg")):
            annotation = ET.parse(path.replace(".jpg",".xml")).getroot()
        elif(path.endswith(".jpeg")):
            annotation = ET.parse(path.replace(".jpeg",".xml")).getroot()
            
        ball_detail = []
        goalpost_detail = []
        robot_detail = []
        
        for element in annotation.findall("object"):
            
            anno_bound_box = element.find("bndbox")
            label = element.find("name").text
            
            if(label == "ball"):
                ball_detail.append(get_bounding_box(anno_bound_box))
            
            elif(label == "goalpost"):
                goalpost_detail.append(get_bounding_box(anno_bound_box))
                
            elif(label == "robot"):
                robot_detail.append(get_bounding_box(anno_bound_box))
                
        dataset = {"image" : image,
                   "ball_detail" : ball_detail,
                   "goalpost_detail" : goalpost_detail,
                   "robot_detail" : robot_detail}
        
        if(self.transform):
            
            if(type(self.transform) is not list):
                self.transform = [self.transform]
            
            for idx in range(len(self.transform)):
                dataset = self.transform[idx](dataset)
                
        return dataset
    

In [4]:
class SegDataset(Dataset):
    
    def __init__(self, path, transform = None):
        
        self.path = path 
        self.transform = transform
        self.file_list = []
        self.target_image = ""
        self.image_path = path + "/image/"
        self.target_path = path + "/target/"

        for file in os.listdir(self.image_path):
            if(file.endswith(".jpg")):
                self.target_image = file.replace(".jpg",".png")
                if os.path.exists(os.path.join(self.target_path, self.target_image)):
                    self.file_list.append(file)
                    
    def __len__(self):
        
        return len(self.file_list)
    
    def __getitem__(self, index):
        
        file_name = self.file_list[index]
        
        image = torch.FloatTensor(self.load_image(self.image_path + "/" + file_name))
        target = torch.FloatTensor(self.load_mask(self.target_path + "/" +  file_name.replace(".jpg",".png")))
        
        return image, target
        
    def load_image(self, path=None):
        
        raw_image = Image.open(path).convert('RGB')
        #raw_image = np.transpose(raw_image.resize((480, 640)), (2,1,0))
        raw_image = F.resize(raw_image, (480, 640))
        #raw_image = F.vflip(raw_image)
        imx_t = np.array(raw_image, dtype=np.float32)/255.0
        imx_t = np.transpose(imx_t, (2, 0, 1))
        return imx_t
    
    def load_mask(self, path=None):
        
        raw_image = Image.open(path)     
        raw_image = np.array(raw_image)
        raw_image[raw_image == 1] = 3    #change ball to ground
        raw_image[raw_image == 2] = 1
        raw_image[raw_image == 3] = 2
#         raw_image = F.resize(Image.fromarray(raw_image), (120, 160))
        imx_t = cv2.resize(raw_image, dsize=(160, 120), interpolation=cv2.INTER_NEAREST)
        imx_t = np.array(imx_t)
#         imx_t[imx_t == 1] = 3    #change ball to ground
#         imx_t[imx_t == 2] = 1
#         imx_t[imx_t == 3] = 2
        #print("Label :  ",np.array(raw_image))
        #imx_t[imx_t==255] = 3
        imx_t = np.transpose(imx_t, (0, 1))
#         imx_t = np.array(raw_image, dtype=np.float32)/255.0
        return imx_t

